<a href="https://colab.research.google.com/github/ddekun/Recommendation_systems/blob/lesson1/lesson1/hw1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [5]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]

In [3]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list_recommended = np.isin(bought_list, recommended_list[:k])
    recall = np.sum(bought_list_recommended) / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)

    recall = recall_at_k(recommended_list, bought_list, k)
    money_spent_recommended = np.sum(prices_recommended[np.isin(recommended_list[:k], bought_list)])
    money_spent_bought = np.sum(prices_bought[np.isin(bought_list, recommended_list[:k])])
    money_recall = recall * (money_spent_recommended / money_spent_bought)
    
    return money_recall

In [6]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [11]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    rr = np.zeros(k)
    for i in range(k):
        if recommended_list[i] in bought_list:
            rr[i] = 1 / (i+1)
    
    mrr = np.sum(rr) / np.min([k, len(bought_list)])
    
    return mrr

In [12]:
mrr_at_k(recommended_list, bought_list, k=5)

0.3125

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [13]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    relevance_scores = np.isin(recommended_list[:k], bought_list).astype(int)
    dcg = np.sum(relevance_scores / np.log2(np.arange(2, k+2)))
    sorted_relevance_scores = np.sort(np.isin(bought_list, recommended_list)[:k])[::-1]
    ideal_dcg = np.sum(sorted_relevance_scores / np.log2(np.arange(2, k+2)))
    ndcg = dcg / ideal_dcg
    
    return ndcg

In [18]:
ndcg_at_k(recommended_list, bought_list, k=4)

0.8772153153380493